In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import json
import re
from datetime import datetime

In [2]:
from selenium.webdriver.edge import service
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from nltk.corpus import stopwords
from selenium.webdriver.common.action_chains import ActionChains

ModuleNotFoundError: No module named 'selenium'

   # Scraping developer stats from Wayback Machine

In [3]:
pages = pd.read_csv("data/safe_obs_dataset.csv", sep=";", index_col = "Unnamed: 0")

In [33]:
pages

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_1,contrlink_2,contrlink_3,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,ext_links_ind
0,/t/app-6-0-compass-of-chance.4440211/,[APP][6.0+]Compass of Chance,"May 2, 2022","\nThe compass does not point to the north, but...",['https://play.google.com/store/apps/details?i...,Baiumka,/m/baiumka.12120441/,New member,"May 2, 2022",4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,https://forum.xda-developers.com/member.php?u=...,https://forum.xda-developers.com/member.php?u=...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,3.0,True,True
4,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,"Oct 25, 2015",\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,https://forum.xda-developers.com/member.php?u=...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,True
7,/t/app-2-2-root-wifi-reaver-gui-for-android.24...,[APP][2.2+][ROOT][WiFi] Reaver-GUI for Android,"Sep 24, 2013",\n\n\n\n\n\nReaver for Android v1.30\nReaver-W...,['https://bcmon.googlecode.com/files/bcmon.apk...,SOEDI,/m/soedi.4096618/,Senior Member,"May 14, 2011",154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
8,/t/app-11-dual-camera-video-recorder.4447581/,[APP][11+] Dual Camera Video Recorder,Today at 8:08 AM,\nCapture video from rear and front cameras at...,['https://play.google.com/store/apps/details?i...,User848,/m/user848.5214037/,Senior Member,"Apr 9, 2013",78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38396,/t/facebook-app-and-chat.678652/,Facebook app and chat,"May 9, 2010",\nI have try to find a real god app to faceboo...,[],tertix,/m/tertix.2493520/,New member,"Apr 11, 2010",4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38406,/t/free-apps-packet-good-apps-packet.678403/,[FREE APPS PACKET] Good apps packet.,"May 9, 2010",\neeehh... sory delete this topic\n \n,[],Cypek,/m/cypek.2556290/,New member,"May 6, 2010",3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38415,/t/full-video-chatt-app-like-skype.675381/,Full Video Chatt App like skype,"May 4, 2010",\nim deploying soon and was wondering if anyon...,[],pyrator,/m/pyrator.1603376/,Senior Member,"Feb 28, 2009",307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True
38435,/t/market-as-separate-app.672499/,Market as separate app,"Apr 29, 2010","\nHey people, I rebooted my phone this morning...",[],fuz1987,/m/fuz1987.2336665/,Member,"Feb 3, 2010",20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,False,True


In [149]:
pages["main_developer_shorturl"] = pages["main_developer_page"].str.strip("/")

In [150]:
members = list(set(pages["main_developer_shorturl"].dropna()))

In [151]:
members

['m/vishnu-pv.5212005',
 'm/startandroid.5748111',
 'm/pettedemon.7205247',
 'm/movm.5674062',
 'm/epicskunk.8154164',
 'm/ajftl.2464901',
 'm/gyebro.4629334',
 'm/morgoo.3491239',
 'm/cold_kitchen.8042046',
 'm/odessyxm.9201459',
 'm/ejatds.4271145',
 'm/amraving.2720678',
 'm/mkrstudio.5467619',
 'm/cyberpedz.2592282',
 'm/pamec.4214344',
 'm/lore-cioni.5980233',
 'm/kenokech2.7085658',
 'm/happylifexda.6143685',
 'm/mustache-man.7097165',
 'm/foolme1729.6255334',
 'm/epicodes.5785480',
 'm/abusedcloth.3982371',
 'm/littlevil.5930923',
 'm/chaikouk.3433891',
 'm/froweey.6087270',
 'm/vipero07.1474981',
 'm/asmodeoux_ru.8298790',
 'm/mobile-idea-studio.5029820',
 'm/grarak.4443334',
 'm/anisbombaywala.4227481',
 'm/stuart-upton666.2749924',
 'm/aqualage.6599716',
 'm/zachionio.9067644',
 'm/magez-it.5842970',
 'm/karolmarusiak.3640408',
 'm/litapp.4691034',
 'm/carlsmileee.5826204',
 'm/skn-xda.4802714',
 'm/diaxotora.11627709',
 'm/mannanshaikh95.5207335',
 'm/reddwall.8093206',
 'm/

In [17]:
"https://forum.xda-developers.com" + pages.loc[,"main_developer_page"]

'https://forum.xda-developers.com/m/user848.5214037/'

In [21]:
wayback_snapshots = ["20201207","20201215", "20201219", "20201224",  "20220101", "20220110", "20220122", "20220123", "20220126", "20220128", "20220204","20220211", "20220223", "20220224","20220301","20220304","20220307","20220308", "20220330", "20220402","20220403","202204415","20220417","20220424","20220429","20220509", "20220510","20220518","20220519","20220526","20220530","20210105", "20210110","20210117", "20210118", "20210123","20210124","20210125","20210126","20210131","20210201", "20210224", "20210307","20210314","20210317","20210319","20210322","20210323","20210324","20210325","20210326","20210330","20210408","20210411","20210421","20210429","20210601","20210608","20210612","20210614","20210801","20210806","20210811","20210812","20210816","20210825","20210902","20210904","20210926","20210929","20211004","20211005","20211006","20211008","20211013","20211015","20211015","20211025","20211027","20211029","20211031","20211105","20211115","20211117","20211209","20211213","20211216","20211217"]

## Checking for available snapshots

In [210]:
try_again = []

In [ ]:
#counter = 0
#snapshots = []
for date in wayback_snapshots[1:]:
    for member in members:
        if member not in empty:
            member_page ={}
            member_page["developer_url"] = member 
            member_page["try_datestamp"] = date
            if member_page.copy() not in finished_dict:
                url = "http://archive.org/wayback/available?url=forum.xda-developers.com/" + member + '&timestamp=' + date
                r = requests.get(url)
                if r:
                    if len(r.json()["archived_snapshots"])>0:
                        if r.json()["archived_snapshots"]["closest"]["status"] == "200":
                            member_page["timestamp"] = r.json()["archived_snapshots"]["closest"]["timestamp"]
                            member_page["url"] = r.json()["archived_snapshots"]["closest"]["url"]
                            check_d = {k: v for k, v in member_page.copy().items() if k != 'try_datestamp'}
                            check_list = [{k: v for k, v in d.items() if k != 'try_datestamp'} for d in snapshots]
                            if check_d.copy() not in check_list:
                                snapshots.append(member_page.copy())
                                print("new")
                    else:
                        empty.append(member)
                        empty = list(set(empty))
                else:
                    try_again.append(member)
                    
    counter += 1
    print(counter)

In [219]:
finished = pd.DataFrame(snapshots)

In [220]:
finished

,developer_url,timestamp,url,try_datestamp
0,m/vishnu-pv.5212005,20201205001500,http://web.archive.org/web/20201205001500/http...,NaN
1,m/movm.5674062,20201219142415,http://web.archive.org/web/20201219142415/http...,NaN
2,m/epicskunk.8154164,20201220155557,http://web.archive.org/web/20201220155557/http...,NaN
3,m/ajftl.2464901,20201225075537,http://web.archive.org/web/20201225075537/http...,NaN
4,m/gyebro.4629334,20201219173848,http://web.archive.org/web/20201219173848/http...,NaN
...,...,...,...,...
12019,m/caliban666.4513035,20220508194756,http://web.archive.org/web/20220508194756/http...,20220123
12020,m/mpiero11.5489276,20220123163642,http://web.archive.org/web/20220123163642/http...,20220123
12021,m/ale82to.5809883,20220126235610,http://web.archive.org/web/20220126235610/http...,20220126
12022,m/zackptg5.6037748,20220213095503,http://web.archive.org/web/20220213095503/http...,20220128


In [222]:
len(snapshots)

12024

In [223]:
finished.loc[finished["try_datestamp"].isnull() == True, "try_datestamp"] = "20201207"

In [224]:
finished.drop_duplicates()

,developer_url,timestamp,url,try_datestamp
0,m/vishnu-pv.5212005,20201205001500,http://web.archive.org/web/20201205001500/http...,20201207
1,m/movm.5674062,20201219142415,http://web.archive.org/web/20201219142415/http...,20201207
2,m/epicskunk.8154164,20201220155557,http://web.archive.org/web/20201220155557/http...,20201207
3,m/ajftl.2464901,20201225075537,http://web.archive.org/web/20201225075537/http...,20201207
4,m/gyebro.4629334,20201219173848,http://web.archive.org/web/20201219173848/http...,20201207
...,...,...,...,...
12019,m/caliban666.4513035,20220508194756,http://web.archive.org/web/20220508194756/http...,20220123
12020,m/mpiero11.5489276,20220123163642,http://web.archive.org/web/20220123163642/http...,20220123
12021,m/ale82to.5809883,20220126235610,http://web.archive.org/web/20220126235610/http...,20220126
12022,m/zackptg5.6037748,20220213095503,http://web.archive.org/web/20220213095503/http...,20220128


In [200]:
finished_dict = finished[["developer_url","try_datestamp"]].to_dict(orient='records')

In [225]:
fin = pd.merge(pages, finished[finished["try_datestamp"]=="20201207"], left_on = "main_developer_shorturl", right_on = "developer_url", how = "outer")

In [227]:
len(empty)

3765

In [208]:
#empty = list(fin[fin["try_datestamp"].isnull()==True]["main_developer_shorturl"])

In [141]:
r.json()["archived_snapshots"]["closest"]

{'status': '200',
 'available': True,
 'url': 'http://web.archive.org/web/20201219080032/https://forum.xda-developers.com/m/scuk.2418057/',
 'timestamp': '20201219080032'}

In [140]:
r.json()["archived_snapshots"]["closest"]["url"]

'http://web.archive.org/web/20201219080032/https://forum.xda-developers.com/m/scuk.2418057/'

In [228]:
finished.to_csv("data/av_dev.csv")

## Scraping the stats

In [5]:
developers = pd.read_csv("data/av_dev.csv", index_col="Unnamed: 0")

In [6]:
developers

,developer_url,timestamp,url,try_datestamp
0,m/vishnu-pv.5212005,20201205001500,http://web.archive.org/web/20201205001500/http...,20201207
1,m/movm.5674062,20201219142415,http://web.archive.org/web/20201219142415/http...,20201207
2,m/epicskunk.8154164,20201220155557,http://web.archive.org/web/20201220155557/http...,20201207
3,m/ajftl.2464901,20201225075537,http://web.archive.org/web/20201225075537/http...,20201207
4,m/gyebro.4629334,20201219173848,http://web.archive.org/web/20201219173848/http...,20201207
...,...,...,...,...
12019,m/caliban666.4513035,20220508194756,http://web.archive.org/web/20220508194756/http...,20220123
12020,m/mpiero11.5489276,20220123163642,http://web.archive.org/web/20220123163642/http...,20220123
12021,m/ale82to.5809883,20220126235610,http://web.archive.org/web/20220126235610/http...,20220126
12022,m/zackptg5.6037748,20220213095503,http://web.archive.org/web/20220213095503/http...,20220128


In [8]:
dev_dict = developers[["developer_url","timestamp","url"]].to_dict(orient = "records")

In [60]:
#counter = 0
for dev in dev_dict[11230:]:
    url = dev["url"]
    r = requests.get(url)
    time.sleep(5)
    if r:
        soup = BeautifulSoup(r.text)
        developer_title = soup.find(class_="userTitle").get_text()
        developer_messages = re.search("\d+", soup.find_all("dd")[0].get_text().replace(",","")).group(0)
        developer_reactions = re.search("\d+", soup.find_all("dd")[1].get_text().replace(",","")).group(0)
        if len(soup.find_all("dd"))>4:
            developer_xp = re.search("\d+", soup.find_all("dd")[2].get_text().replace(",","")).group(0)
            developer_achievements = re.search("\d+", soup.find_all("dd")[3].get_text().replace(",","")).group(0)
            developer_level = re.search("\d+", soup.find_all("dd")[4].get_text().replace(",","")).group(0)
            dev["developer_xp"] = developer_xp
            dev["developer_achievements"] = developer_achievements
            dev["developer_level"] = developer_level
        dev["developer_title"] = developer_title
        dev["developer_messages"] =developer_messages
        dev["developer_reactions"]= developer_reactions
    counter += 1
    if counter % 100 == 0:
        print(counter)


10700
10800
10900
11000
11100
11200
11300
11400


In [61]:
finished = pd.DataFrame(dev_dict)

In [62]:
finished.loc[finished["developer_title"].isnull()==False]

,developer_url,timestamp,url,developer_xp,developer_achievements,developer_level,developer_title,developer_messages,developer_reactions
0,m/vishnu-pv.5212005,20201205001500,http://web.archive.org/web/20201205001500/http...,0,8,25,Recognized Themer,1182,2078
1,m/movm.5674062,20201219142415,http://web.archive.org/web/20201219142415/http...,0,17,21,Senior Member,388,62
2,m/epicskunk.8154164,20201220155557,http://web.archive.org/web/20201220155557/http...,0,22,4,Member,11,0
3,m/ajftl.2464901,20201225075537,http://web.archive.org/web/20201225075537/http...,0,30,30,Senior Member,2160,135
4,m/gyebro.4629334,20201219173848,http://web.archive.org/web/20201219173848/http...,0,6,23,Senior Member,124,471
...,...,...,...,...,...,...,...,...,...
12019,m/caliban666.4513035,20220508194756,http://web.archive.org/web/20220508194756/http...,14571,82,38,Senior Member,6846,4340
12020,m/mpiero11.5489276,20220123163642,http://web.archive.org/web/20220123163642/http...,4996,16,25,Senior Member,152,32
12021,m/ale82to.5809883,20220126235610,http://web.archive.org/web/20220126235610/http...,6481,28,28,Senior Member,378,79
12022,m/zackptg5.6037748,20220213095503,http://web.archive.org/web/20220213095503/http...,12611,59,36,Recognized Developer,4109,6852


In [64]:
finished.to_csv("data/WBdev_batch0.csv", sep =";")

In [111]:
scraped = pd.DataFrame(snapshots)